# Buisness Context & Objectives

Currently shareholders at the telecom company are concerned about declining customer retention and its effect on recurring revenue

##### **Key metrics to track**
- Overall churn rate
- Monthly recurring revenue (MRR) at risk
- Customer lifetime value (CLV)
- High-value customer retention rates
- Churn rates by customer segment

##### **Questions to Answer:**
1. Who is churning and what segments are most at risk?
2. Why are customers leaving — what patterns emerge?
3. What is the financial impact of churn?
4. Which retention strategies give the best ROI?
5. Can we predict future churn to target customers before they leave?

##### **Our goal is to deliver:**
- A churn risk model
- Clear insights into churn drivers
- Data-driven recommendations to increase retention and profitability


# Data Cleaning

In [95]:
import pandas as pd

In [96]:
churn_df = pd.read_csv('data\Telco-Customer-Churn.csv')

### Understading the Structure

In [97]:
#shape of the dataset
print(churn_df.shape)
#columns of the dataset
print(churn_df.columns)
#first 5 rows of the dataset
churn_df.head()

(7043, 21)
Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [98]:
# Getting the data types of the columns
print(churn_df.dtypes)

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object


### Fixing data Type Issues

In [99]:
# Total Charges is string but it needs to be a float
new_total_charges = []
for charge in churn_df['TotalCharges']:
    if charge == ' ':
        new_total_charges.append(0.0)
    else:
        new_total_charges.append(float(charge))
churn_df['TotalCharges'] = new_total_charges

In [100]:
# Dropping rows with 0.0 values in TotalCharges, Won't have impact on analysis because it only makes up 11 rows
churn_df[churn_df['TotalCharges'] == 0.0].index
churn_df.drop(churn_df[churn_df['TotalCharges'] == 0.0].index, inplace=True)

### Standardize Categorical Variables

In [101]:
# Replacing 'No phone service' with 'No' in the MultipleLines column
churn_df['MultipleLines'] = churn_df['MultipleLines'].replace('No phone service', 'No')
# Replacing Senior Citizen with 'Yes' and 'No'
churn_df['SeniorCitizen'] = churn_df['SeniorCitizen'].replace({1: 'Yes', 0: 'No'})

### Creating new features

In [102]:
# Tenure groupings
def tenure_group(tenure):
    if tenure <= 12:
        return '0-12'
    elif 12 < tenure <= 24:
        return '13-24'
    elif 24 < tenure <= 36:
        return '25-36'
    elif 36 < tenure <= 48:
        return '37-48'
    else:
        return '49+'
churn_df['TenureGroup'] = churn_df['tenure'].apply(tenure_group)

In [103]:
# Revenue Related Features
def total_revenue(row):
    return round(row['MonthlyCharges'] * row['tenure'], 2)
def average_charges(row):
    return round(row['TotalCharges'] / row['tenure'] if row['tenure'] > 0 else 0, 2)

churn_df['TotalRevenue'] = churn_df.apply(total_revenue, axis=1)
churn_df['AverageCharges'] = churn_df.apply(average_charges, axis=1)

In [104]:
churn_df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,TenureGroup,TotalRevenue,AverageCharges
0,7590-VHVEG,Female,No,Yes,No,1,No,No,DSL,No,...,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,0-12,29.85,29.85
1,5575-GNVDE,Male,No,No,No,34,Yes,No,DSL,Yes,...,No,One year,No,Mailed check,56.95,1889.50,No,25-36,1936.30,55.57
2,3668-QPYBK,Male,No,No,No,2,Yes,No,DSL,Yes,...,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,0-12,107.70,54.08
3,7795-CFOCW,Male,No,No,No,45,No,No,DSL,Yes,...,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,37-48,1903.50,40.91
4,9237-HQITU,Female,No,No,No,2,Yes,No,Fiber optic,No,...,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,0-12,141.40,75.83


### Making variables better for modeling

In [112]:
#creating a copy of the churn_df for modeling purposes
churn_model_df = churn_df.copy()

In [113]:
# Column Transformations, 1 being Yes and 0 being No
churn_model_df['SeniorCitizen'] = churn_model_df['SeniorCitizen'].replace({'Yes': 1, 'No': 0})
churn_model_df['Partner'] = churn_model_df['Partner'].replace({'Yes': 1, 'No': 0})
churn_model_df['Dependents'] = churn_model_df['Dependents'].replace({'Yes': 1, 'No': 0})
churn_model_df['PhoneService'] = churn_model_df['PhoneService'].replace({'Yes': 1, 'No': 0})
churn_model_df['MultipleLines'] = churn_model_df['MultipleLines'].replace({'Yes': 1, 'No': 0})
churn_model_df['OnlineSecurity'] = churn_model_df['OnlineSecurity'].replace({'Yes': 1, 'No': 0, 'No internet service': 0})
churn_model_df['OnlineBackup'] = churn_model_df['OnlineBackup'].replace({'Yes': 1, 'No': 0, 'No internet service': 0})
churn_model_df['DeviceProtection'] = churn_model_df['DeviceProtection'].replace({'Yes': 1, 'No': 0, 'No internet service': 0})
churn_model_df['TechSupport'] = churn_model_df['TechSupport'].replace({'Yes': 1, 'No': 0, 'No internet service': 0})
churn_model_df['StreamingTV'] = churn_model_df['StreamingTV'].replace({'Yes': 1, 'No': 0, 'No internet service': 0})
churn_model_df['StreamingMovies'] = churn_model_df['StreamingMovies'].replace({'Yes': 1, 'No': 0, 'No internet service': 0})
churn_model_df['PaperlessBilling'] = churn_model_df['PaperlessBilling'].replace({'Yes': 1, 'No': 0})
churn_model_df['Churn'] = churn_model_df['Churn'].replace({'Yes': 1, 'No': 0})


C:\Users\djhic\AppData\Local\Temp\ipykernel_64360\1585173383.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  churn_model_df['SeniorCitizen'] = churn_model_df['SeniorCitizen'].replace({'Yes': 1, 'No': 0})
C:\Users\djhic\AppData\Local\Temp\ipykernel_64360\1585173383.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  churn_model_df['Partner'] = churn_model_df['Partner'].replace({'Yes': 1, 'No': 0})
C:\Users\djhic\AppData\Local\Temp\ipykernel_64360\1585173383.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated an

In [114]:
churn_model_df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,TenureGroup,TotalRevenue,AverageCharges
0,7590-VHVEG,Female,0,1,0,1,0,0,DSL,0,...,0,Month-to-month,1,Electronic check,29.85,29.85,0,0-12,29.85,29.85
1,5575-GNVDE,Male,0,0,0,34,1,0,DSL,1,...,0,One year,0,Mailed check,56.95,1889.50,0,25-36,1936.30,55.57
2,3668-QPYBK,Male,0,0,0,2,1,0,DSL,1,...,0,Month-to-month,1,Mailed check,53.85,108.15,1,0-12,107.70,54.08
3,7795-CFOCW,Male,0,0,0,45,0,0,DSL,1,...,0,One year,0,Bank transfer (automatic),42.30,1840.75,0,37-48,1903.50,40.91
4,9237-HQITU,Female,0,0,0,2,1,0,Fiber optic,0,...,0,Month-to-month,1,Electronic check,70.70,151.65,1,0-12,141.40,75.83


In [115]:
# One Hot Encoding for categorical variables
categorical_columns = ['InternetService', 'Contract', 'PaymentMethod']
churn_model_df = pd.get_dummies(data=churn_model_df, columns=categorical_columns, drop_first=True)

# Changing new columns to 0 and 1 instead of True and False
for column in churn_model_df.columns:
    if churn_model_df[column].dtype == 'bool':
        churn_model_df[column] = churn_model_df[column].astype(int)

In [116]:
churn_model_df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,...,TenureGroup,TotalRevenue,AverageCharges,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,7590-VHVEG,Female,0,1,0,1,0,0,0,1,...,0-12,29.85,29.85,0,0,0,0,0,1,0
1,5575-GNVDE,Male,0,0,0,34,1,0,1,0,...,25-36,1936.30,55.57,0,0,1,0,0,0,1
2,3668-QPYBK,Male,0,0,0,2,1,0,1,1,...,0-12,107.70,54.08,0,0,0,0,0,0,1
3,7795-CFOCW,Male,0,0,0,45,0,0,1,0,...,37-48,1903.50,40.91,0,0,1,0,0,0,0
4,9237-HQITU,Female,0,0,0,2,1,0,0,0,...,0-12,141.40,75.83,1,0,0,0,0,1,0


### Uploading Dataframes to CSV in new 'CleanedData' folder

In [117]:
churn_df.to_csv('CleanedData\churn_df.csv', index=False)
churn_model_df.to_csv('CleanedData\churn_model_df.csv', index=False)